In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from env import user, password, host
from scipy.stats import levene, ttest_ind
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
import math
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import statsmodels.api as sm
import wrangle_campus as wc
import warnings
warnings.filterwarnings("ignore")

In [2]:
cs22 = pd.read_csv('CAMPUS_summary_22.csv')
cs21 = pd.read_csv('CAMPUS_summary_21.csv')
cs20 = pd.read_csv('CAMPUS_summary_20.csv')
cs19 = pd.read_csv('CAMPUS_summary_19.csv')
cs18 = pd.read_csv('CAMPUS_summary_18.csv')

In [3]:
cs22=wc.campus_prep(cs22)
cs21=wc.campus_prep(cs21)
cs20=wc.campus_prep(cs20)
cs19=wc.campus_prep(cs19)
cs18=wc.campus_prep(cs18)

In [4]:
df=wc.df_combine(cs18,cs19,cs20,cs21,cs22)

In [5]:
df

,campus_name_num,charter_encoded,student_enrollment,discipline_count,discipline_percent
0,CAYUGA MIDDLE 001902041,0,155.0,38.0,25.0
1,ELKHART H S 001903001,0,411.0,205.0,50.0
2,ELKHART MIDDLE 001903041,0,298.0,151.0,51.0
3,ELKHART EL 001903101,0,379.0,11.0,3.0
4,ELKHART INT 001903102,0,276.0,151.0,55.0
...,...,...,...,...,...
28421,ZAPATA H S 253901001,0,1028.0,239.0,23.0
28422,ZAPATA MIDDLE 253901041,0,856.0,298.0,35.0
28423,ZAPATA SOUTH EL 253901106,0,516.0,13.0,3.0
28424,CRYSTAL CITY H S 254901001,0,556.0,310.0,56.0


In [7]:
train, validate, test= wc.split_campus_data(df)

In [8]:
train.head()

,campus_name_num,charter_encoded,student_enrollment,discipline_count,discipline_percent
14461,IRVIN H S 071902008,0,1300.0,159.0,12.0
15829,VETERANS MIDDLE 108902046,0,949.0,1163.0,123.0
21119,LORENA HIGH 161907001,0,530.0,71.0,13.0
9136,TIPPS EL 101907141,0,1022.0,120.0,12.0
13930,YOUNG WOMEN'S STEAM ACADEMY AT BAL 057905352,0,1290.0,97.0,8.0


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15918 entries, 14461 to 12928
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   campus_name_num     15918 non-null  object 
 1   charter_encoded     15918 non-null  int64  
 2   student_enrollment  15918 non-null  float64
 3   discipline_count    15918 non-null  float64
 4   discipline_percent  15918 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 746.2+ KB


In [10]:
validate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6822 entries, 5913 to 4436
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   campus_name_num     6822 non-null   object 
 1   charter_encoded     6822 non-null   int64  
 2   student_enrollment  6822 non-null   float64
 3   discipline_count    6822 non-null   float64
 4   discipline_percent  6822 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 319.8+ KB


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5686 entries, 25630 to 18794
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   campus_name_num     5686 non-null   object 
 1   charter_encoded     5686 non-null   int64  
 2   student_enrollment  5686 non-null   float64
 3   discipline_count    5686 non-null   float64
 4   discipline_percent  5686 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 266.5+ KB
